**1. Geliştirme Ortamı (Kurulum)**

Proje için gerekli tüm kütüphaneler yüklenmektedir.
LangChain, FAISS, Gemini API ve Gradio bileşenleri RAG tabanlı chatbot için gereklidir.

In [ ]:
!pip install -q google-generativeai langchain langchain-community faiss-cpu sentence-transformers langchain-google-genai datasets huggingface_hub pyarrow gradio


**2. API ve Modül Kurulumu**

Bu hücrede gerekli modüller import edilmekte ve Google Gemini ile Hugging Face API anahtarları tanımlanmaktadır.
API’ler, modelin metin üretimi ve embedding oluşturması için kullanılır.

In [2]:
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from datasets import load_dataset
from google.colab import userdata
import gradio as gr
import os

try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    HF_TOKEN = userdata.get('HF_TOKEN')
except:
    print("Secrets bulunamadı")

os.environ['HUGGINGFACE_HUB_TOKEN'] = HF_TOKEN
genai.configure(api_key=GOOGLE_API_KEY)


**3. Veri Seti Hazırlama**

Bu hücrede “aliarda/Turkish-Poems-19K” adlı Türkçe şiir veri seti Hugging Face’ten yüklenmektedir.
siir_metni sütunundaki ilk 1000 şiir alınarak modelin öğrenmesi için kullanılmaktadır.

In [ ]:
dataset = load_dataset("aliarda/Turkish-Poems-19K", token=HF_TOKEN)
print(f"Veri seti yüklendi! Toplam {len(dataset['train'])} şiir bulundu.")

poems = [
    item["siir_metni"].strip()
    for item in dataset["train"]
    if item["siir_metni"] and isinstance(item["siir_metni"], str) and item["siir_metni"].strip()
]
poems = poems[:1000]
print(f"{len(poems)} şiir işleme alındı.")

**4. Embedding ve Vektör Veritabanı (FAISS)**

Bu aşamada şiirler embedding (vektör temsili) haline getirilir ve FAISS veritabanına kaydedilir.
FAISS, metin benzerliğini hızlı hesaplamak için kullanılır.
Her şiir parçası 300 karakterlik küçük bölümlere ayrılır.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)

docs = []
for p in poems:
    if p and isinstance(p, str) and p.strip():
        docs.extend(text_splitter.create_documents([p.strip()]))

vectorstore = FAISS.from_documents(docs, embeddings)
print("✅ FAISS veritabanı oluşturuldu!")

**5. RAG Zinciri ve LLM Modeli**

Bu bölümde Gemini 2.5 Pro modeli kullanılarak LangChain tabanlı bir RAG zinciri oluşturulur.
Kullanıcının girdiği temaya göre, veri tabanındaki benzer şiirlerden esinlenerek yeni bir şiir üretilir.

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.8
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

print("Sistem hazır! Şiir üretmeye hazırsınız")


**6. Gradio Arayüzü**

Bu hücre, Gradio kullanarak etkileşimli bir web arayüzü oluşturur.
Kullanıcı şiir temasını yazar, sistem RAG modeliyle yeni bir Türkçe şiir üretir.

In [ ]:
def siir_uretici(tema):
    prompt = f"""Türkçe, sanatsal ve kafiyeli bir şiir yaz.
    Şiir şu temaya uygun olmalı: '{tema}'.
    Veri tabanındaki Türkçe şiirlerin tarzından esinlen:
    Şiir özellikleri:
    - Her dize kısa olsun (4–8 kelime arası)
    - Son kelimelerde kafiye uyumu bulunsun
    - Gerekirse iç uyak (iç kafiye) da kullanılabilir
    - Akıcı, duygusal ve imgelerle dolu olsun
    - Doğa ve insan temaları kullanılabilir
    - Sadece şiiri yaz, açıklama yapma.
    """

    response = qa_chain.invoke({"question": prompt, "chat_history": []})
    return response["answer"].strip()


demo = gr.Interface(
    fn=siir_uretici,
    inputs=gr.Textbox(
        label="🎭 Şiir Teması",
        placeholder="Örnek: Ayrılık, doğa, aşk...",
        lines=1
    ),
    outputs=gr.Textbox(
        label="Üretilen Şiir",
        lines=10,
        max_lines=20
    ),
    title="Türkçe Şiir Üretici",
    theme="soft",
    allow_flagging="never"
)

demo.launch(share=True)